# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
alt.data_transformers.disable_max_rows()
alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [11]:
start_date = pd.to_datetime("2020-04-13")

In [12]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

,poblacion,casos_confirmados,fecha,region_id,region,provincia_id,provincia,comuna_id,comuna,tasa
0,247552.0,270.0,2020-04-27,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,109.067994
1,247552.0,9138.0,2020-10-12,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,3691.345657
2,247552.0,596.0,2020-05-29,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,240.757497
3,247552.0,6131.0,2020-08-17,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,2476.651370
4,247552.0,353.0,2020-05-11,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,142.596303


In [13]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

,region,codigo_region,comuna,codigo_comuna,poblacion,fecha,casos_activos
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-04-13,88.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-04-13,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-04-13,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-04-13,0.0
4,Tarapaca,1,Alto Hospicio,1107.0,129999.0,2020-04-13,8.0


In [14]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

,region,fecha,fallecidos
374,Arica y Parinacota,2020-04-13,1.0
375,Tarapacá,2020-04-13,0.0
376,Antofagasta,2020-04-13,1.0
377,Atacama,2020-04-13,0.0
378,Coquimbo,2020-04-13,0.0


In [15]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.tail()

,grupo_de_edad,fecha,fallecidos
1654,50-59,2020-12-01,1547
1655,60-69,2020-12-01,3256
1656,70-79,2020-12-01,4337
1657,80-89,2020-12-01,3987
1658,>=90,2020-12-01,1434


## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos a la fecha por cada grupo etáreo.

In [16]:
#fallecidos_etareo.iloc[-7:,:] Entrega un dataframe correspondiente a los datos de la ultima fecha registrada.

In [17]:
alt.Chart(fallecidos_etareo.iloc[-7:,:]).mark_bar().encode(
    x = alt.X('grupo_de_edad:O', title =  'Grupo etáreo'),
    y = alt.Y('fallecidos:Q', title = 'Fallecidos'),
    tooltip = [alt.Tooltip('fecha:T', title = 'Fecha'),
               alt.Tooltip('fallecidos:Q', title = 'Fallecidos'),
               alt.Tooltip('grupo_de_edad:O', title = 'Grupo etáreo')
    ]
).properties(title = 'Fallecidos a la fecha por grupo etáreo',
             width = 700,
             height= 300
)

alt.Chart(...)

**Comentarios:** Podemos notar claramente como los fallecimientos por covid-19 se concentran en las personas de tercera edad, en especial a los del grupo etáteo 70-79. También es posible visualizar que son los jovenes quienes han fallecido menos.

## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [9]:
alt.Chart(activos).mark_bar().encode(
    x = alt.X('comuna:O', title = 'Comuna', sort= '-y'),
    y = alt.Y('poblacion:Q', title = 'Población'),
    tooltip = [alt.Tooltip('comuna', title = 'Comuna'),
               alt.Tooltip('poblacion', title = 'Población')
    ]
).properties(title = 'Población por comuna'
)

alt.Chart(...)

**Comentarios:** Gracias al grafico de barra podemos darnos cuenta que la gran mayoria de las comunas tienen una población inferior a las 200.000 personas y son muy pocas las comunas que superan las 400.000 personas. También podemos señalar que Puente Alto es la comuna con mayor cantidad de población, mientras que la Antartica es la comuna con menor cantidad.

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [10]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['fecha'], empty='none')

# The basic line
line = alt.Chart(fallecidos_etareo).mark_line(interpolate='basis').encode(
    x= alt.X('fecha:T', title = 'Fecha'),
    y= alt.Y('fallecidos', title = 'Fallecidos'),
    color= alt.Color('grupo_de_edad', title = 'Grupo etáreo', scale = alt.Scale(scheme = 'dark2')),
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(fallecidos_etareo).mark_point().encode(
    x='fecha:T',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-12).encode(
    text=alt.condition(nearest, 'fallecidos:Q', alt.value(' '))
)

text2 = line.mark_text(align='left', dx=-80, dy=-12).encode(
    text=alt.condition(nearest, 'fecha:T', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(fallecidos_etareo).mark_rule(color='gray').encode(
    x='fecha:T',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text, text2
).properties(title = 'Fallecidos a la fecha por grupo etáreo',
             width = 700,
             height= 300
)

alt.LayerChart(...)

**Comentarios:** Podemos darnos cuenta como a principios de Mayo las distintas curvas correspondiente a los diferentes grupos etáreos comienzan a aumentar a diferente velocidad. Notemos como los grupos etáreos siempre mantienen el mismo orden de fallecidos para cualquier fecha a partir de Mayo (desde mediados de mayo podemos comenzar a diferenciar las distintas curvas), por ejemplo el grupo etáreo con mayor cantidad de fallecidos es siempre 70-79, mientras que el de menor cantidad es siempre 0-39.

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [11]:
dataframe1 = confirmados.groupby('fecha')['casos_confirmados'].sum().reset_index()
dataframe2 = fallecidos.groupby('fecha')['fallecidos'].sum().reset_index()
dataframe3 = activos.groupby('fecha')['casos_activos'].sum().reset_index()

In [12]:
grafico1 = alt.Chart(dataframe1).mark_line().encode(
    x = alt.X('fecha:T', title =  'Fecha'),
    y = alt.Y('casos_confirmados:Q', title = 'Casos confirmados'),
    tooltip = [alt.Tooltip('fecha:T', title = 'Fecha'),
               alt.Tooltip('casos_confirmados', title = 'Casos confirmados hasta la fecha')
    ]
).properties(title = 'Evolución de casos confirmados',
             width = 700,
             height= 300
)

grafico2 = alt.Chart(dataframe2).mark_line().encode(
    x = alt.X('fecha:T', title = 'Fecha'), 
    y = alt.Y('fallecidos:Q', title = 'Fallecidos'),
    tooltip = [alt.Tooltip('fecha:T', title = 'Fecha'),
               alt.Tooltip('fallecidos:Q', title = 'Casos fallecidos hasta la fecha')
    ] 
).properties(title = 'Evolución de fallecimientos',
             width = 700,
             height= 300
)

grafico3 = alt.Chart(dataframe3).mark_line().encode(
    x = alt.X('fecha:T', title = 'Fecha'), 
    y = alt.Y('casos_activos:Q', title = 'Casos activos'),
    tooltip = [alt.Tooltip('fecha:T', title = 'Fecha'),
               alt.Tooltip('casos_activos:Q', title = 'Casos activos')
    ]
).properties(title = 'Evolución de casos activos',
             width = 700,
             height= 300
)

grafico1 & grafico2 & grafico3


alt.VConcatChart(...)

**Comentarios:** Podemos notar claramente como la gráfica "Evolución de casos confirmados" y "Evolución de fallecimientos" tienen una correlación (gráficas muy similares), mientras que la gráfica "Evolución de casos activos" es muy diferente a los otros dos gráficos. Las pendientes de las primeras dos curvas son siempre positiva, mientras que en la tercera tenemos pendientes positivas, negativas e iguales a cero. Es importante señalar que estas gráficas tienen el mismo eje "x", pero el eje "y" de las gráficas estan en diferentes escalas, es decir, a pesar de que las dos primeras gráficas se ven similares a simple vista, son en realidad muy diferentes ya que por ejemplo el valor maximo de uno es 600.000 mientras que en la otra es solo de 30.000

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [13]:
dataframe = confirmados.groupby(['fecha', 'region'])['tasa'].sum().reset_index()

In [14]:
alt.Chart(dataframe).mark_line().encode(
    x= alt.X('fecha:T', title = 'Fecha'),
    y= alt.Y('tasa:Q', title = 'Tasa'),
    color = alt.Color('region', title = 'Regiones'),
    tooltip = [alt.Tooltip('fecha', title = 'Fecha'),
               alt.Tooltip('tasa', title = 'Tasa'),
               alt.Tooltip('region', title= 'Región')
    ]
).properties(title = 'Tasa de incidencia entre regiones a lo largo del tiempo',
             width = 900,
             height= 300
)

alt.Chart(...)

**Comentarios:** Notemos que todas las regiones, a expeción de la Metropolitana, tienen una tasa de incidencia inferior a 80.000. Al haber tantas regiones dificulta un poco la designación de colores, por lo tanto tenemos que tener cuidado con confundir colores. Podemos darnos cuenta que en Octubre muchas de estas curvas se intersectan. Es posible notar que la región de "Magallanes y la Antártica Chilena" a mediados de agosto comienza a tener un gran aumento en su tasa de incidencia la cual comienza a normalizarce a mediados de Octubre.

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [15]:
date1 = pd.to_datetime("2020-04-13")
date2 = pd.to_datetime("2020-11-06")

mask1 = (confirmados['fecha'] == date1) | (confirmados['fecha'] == date2)

dataframe1 = confirmados.loc[mask1]
dataframe1.head()


,poblacion,casos_confirmados,fecha,region_id,region,provincia_id,provincia,comuna_id,comuna,tasa
19,247552.0,10103.0,2020-11-06,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,4081.162746
31,247552.0,115.0,2020-04-13,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,46.454886
91,1233.0,28.0,2020-11-06,15.0,Arica y Parinacota,151.0,Arica,15102.0,Camarones,2270.884023
103,1233.0,0.0,2020-04-13,15.0,Arica y Parinacota,151.0,Arica,15102.0,Camarones,0.000000
163,810.0,64.0,2020-11-06,15.0,Arica y Parinacota,152.0,Parinacota,15202.0,General Lagos,7901.234568


In [16]:
alt.Chart(dataframe1).mark_circle().encode(
    x = alt.X('casos_confirmados', title = 'Casos confirmados'),
    y = alt.Y('tasa', title = 'Tasa'),
    color = alt.Color('region', title = 'Regiones'),
    tooltip = [alt.Tooltip('comuna', title = 'Comuna'),
               alt.Tooltip('casos_confirmados', title = 'Casos confirmados'),
               alt.Tooltip('tasa', title = 'Tasa'),
               alt.Tooltip('fecha', title = 'Fecha')
    ]
).properties(title = 'Tasa v/s Casos confirmados',
             width = 700,
             height = 300)

alt.Chart(...)

In [17]:
alt.Chart(dataframe1).mark_circle().encode(
    x = alt.X('casos_confirmados', title = 'Casos confirmados'),
    y = alt.Y('tasa', title = 'Tasa'),
    color = alt.Color('region', title = 'Regiones'),
    tooltip = [alt.Tooltip('comuna', title = 'Comuna'),
               alt.Tooltip('casos_confirmados', title = 'Casos confirmados'),
               alt.Tooltip('tasa', title = 'Tasa'),
               alt.Tooltip('fecha', title = 'Fecha')
    ],
    size = 'poblacion'
).properties(title = 'Tasa v/s Casos confirmados',
             width = 700,
             height = 300)

alt.Chart(...)

**Comentarios:** Gracias a los datos que se muestran cuando el cursor esta sobre un punto podemos darnos cuenta que los puntos mas lejanos del (0,0) corresponden al 6 de noviembre, mientras que los datos del 13 de abril estan muy concentrados y cercanos a este punto, dificultado la distincion entre uno y otro. Además considero que si es útil que el tamaño sea proporcinal a la población. Esto nos entregaria más información para hacer un analisis más profundo.

Como vemos en el segundo gráfico, podemos darnos cuenta rapidamente que las comunas que tiene una población mas grande tienden a tener una mayor cantidad de casos confirmados.

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [18]:
alt.Chart(activos).mark_line().encode(
    x = alt.X('fecha:T', title =  'Fecha'),
    y = alt.Y('casos_activos:Q', title = 'Casos activos'),
    color = alt.Color('comuna:O', title = 'Grupo etáreo'),
    tooltip = [alt.Tooltip('fecha:T', title = 'Fecha'),
               alt.Tooltip('casos_activos', title = 'Casos activos'),
               alt.Tooltip('comuna', title = 'Comuna')
    ]
).properties(title = 'Evolución de casos activos por comuna',
             width = 700,
             height= 300
)

alt.Chart(...)

In [19]:
alt.Chart(activos).mark_line().encode(
    x = alt.X('fecha:T', title =  'Fecha'),
    y = alt.Y('casos_activos:Q', title = 'Casos activos'),
    color = alt.Color('comuna:O', title = 'Comunas', scale = alt.Scale(scheme = 'category20b')),
    tooltip = [alt.Tooltip('fecha:T', title = 'Fecha'),
               alt.Tooltip('casos_activos', title = 'Casos activos'),
               alt.Tooltip('region', title = 'Region'),
               alt.Tooltip('comuna', title = 'Comuna')
    ],
    row = alt.Row('region:O', title = 'Regiones', header = alt.Header(labelFontSize=13))
).properties(title = alt.TitleParams(text = 'Evolución de casos activos por comuna (separados por región)', dx = 260, dy = -10),
             width = 700,
             height= 300
).resolve_scale(x = 'independent', y = 'independent', color = 'independent')


alt.Chart(...)

**Pros y contras del primer enfoque:**
* Al haber muchas comunas en el pais es dificil designarle un color a cada una para diferenciarlas entre una y otra.
* Podemos darnos una idea global del estado del pais a lo largo del tiempo.

**Pros y contras del segundo enfoque:**
* Podemos hacer un analisis más profundo con este enfoque. Al haber menos comunas por grafico resulta más facil diferenciar entre una y otra (en algúnas regiones se sigue dificultando, pero menos que antes).
* Podemos identificar rapidamente si hay alguna comuna que sobresalga ante las demas de la misma región. Es decir, si hay alguna comuna dentro de una región que tenga un alto nivel de casos activos, será facil de detectar.
* Podemos identificar las distintas fechas donde las regiones tuvieron una alza o baja de casos confirmados.

**Observación:** En los gráficos del segundo enfoque hay que notar que la escala del eje "casos confirmados" es diferente para cada región.

## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

__Observacion:__ Si descomentamos las lineas siguientes podemos darnos cuenta que en los tres dataframe a usar las regiones estan escrita de diferentes formas, es por esto que se crearan dos diccionarios para reemplazar los nombres de las regiones en los conjuntos de datos para poder unirlos.


In [20]:
#activos['region'].unique()
#confirmados['region'].unique()
#fallecidos['region'].unique()

In [21]:
region_names1 = {
    'Aysen' : 'Aysén' ,
    'Biobio' : 'Biobío' ,
    'Del Libertador General Bernardo O’Higgins' : 'O’Higgins' ,
    'La Araucania' : 'Araucanía',
    'Los Rios' : 'Los Ríos',
    'Magallanes y la Antartica' : 'Magallanes',
    'Nuble' : 'Ñuble',
    'Tarapaca' : 'Tarapacá',
    'Valparaiso' : 'Valparaíso'
    
}

region_names2 = {
    'Aysén del General Carlos Ibáñez del Campo' : 'Aysén',
    'La Araucanía' : 'Araucanía',
    "Libertador General Bernardo O'Higgins" : 'O’Higgins',
    'Magallanes y de la Antártica Chilena' : 'Magallanes',
    'Metropolitana de Santiago' : 'Metropolitana'
}

In [22]:
df_nuevo1 = pd.DataFrame(activos.groupby('region')['casos_activos'].mean()).reset_index()
df_nuevo1 = df_nuevo1.replace(region_names1)

df_nuevo2 = pd.DataFrame(confirmados.groupby('region')['casos_confirmados'].max()).reset_index()
df_nuevo2 = df_nuevo2.replace(region_names2)

df_nuevo3 = pd.DataFrame(fallecidos.groupby('region')['fallecidos'].max()).drop('Total', axis = 0).reset_index()

df_nuevo = pd.merge(df_nuevo1, df_nuevo2, on = 'region')
df_nuevo = pd.merge(df_nuevo, df_nuevo3, on = 'region')

df_nuevo.columns = ['region', 'mean_casos_activos', 'max_casos_confirmados', 'max_fallecidos']

df_nuevo.head()

,region,mean_casos_activos,max_casos_confirmados,max_fallecidos
0,Antofagasta,78.112795,12831.0,562.0
1,Arica y Parinacota,100.310606,10530.0,207.0
2,Atacama,33.713805,4820.0,106.0
3,Aysén,4.406061,1086.0,14.0
4,Biobío,46.290174,6401.0,607.0


In [23]:
grafico1 = alt.Chart(df_nuevo).mark_circle(size = 30).encode(
    x = alt.X('region:O', title = 'Regiones'),
    y = alt.Y('mean_casos_activos:Q', title = 'Personas'),
    color = alt.value('steelblue'),
    tooltip = [alt.Tooltip('mean_casos_activos', title = 'Promedio de casos activos'),
               alt.Tooltip('region', title = 'Region')
              ]
)

grafico2 = alt.Chart(df_nuevo).mark_circle(size = 30).encode(
    x = alt.X('region:O', title = 'Regiones'),
    y = alt.Y('max_casos_confirmados:Q', title = 'Personas'),
    color = alt.value('green'),
    tooltip = [alt.Tooltip('max_casos_confirmados', title = 'Maximo de casos confirmados'),
               alt.Tooltip('region', title = 'Region')
              ]
)

grafico3 = alt.Chart(df_nuevo).mark_circle(size = 30).encode(
    x = alt.X('region:O', title = 'Regiones'),
    y = alt.Y('max_fallecidos:Q', title = 'Personas'),
    color = alt.value('darkred'),
    tooltip = [alt.Tooltip('max_fallecidos', title = 'Maximo de fallecidos'),
               alt.Tooltip('region', title = 'Region')
              ]
)


alt.layer(
    grafico1, grafico2, grafico3
).properties(title = 'Promedio de casos activos, Maximo de casos confirmados y Maximo de fallecidos por región',
             width = 700,
             height= 300
)

alt.LayerChart(...)

**Comentarios:** No supe como poner las leyendas en este caso, es decir, una zona informativa en el lado derecho del gráfico que diera a conocer a que corresponde cada color, es por esto que se usaron los "Tooltip" para que esta información del color sea clara al poner el cursos sobre los puntos.

Podemos notar claramente que el maximo de casos confirmados es mayor que el maxmo de fallecidos y que el promedio de casos activos.

La región Metropolitana tiene el mayor maximo de casos confirmados y maximo de fallecidos. Mientras que Aysén es la region con el menor máximo de casos confirmados.

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [24]:
import geopandas as gpd
from pathlib import Path

In [25]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.head()

,Region,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_,geometry
0,Región Metropolitana de Santiago,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,"POLYGON ((-7873736.745 -3885505.642, -7873695...."
1,Región de Antofagasta,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,Región de Arica y Parinacota,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,"POLYGON ((-7727277.278 -1997230.768, -7726464...."
3,Región de Atacama,1089,4,3,75661.248635,9.643906e+10,2.401741e+06,"MULTIPOLYGON (((-7900342.628 -3153340.296, -79..."
4,Región de Aysén del Gral.Ibañez del Campo,1088,14,11,106703.377369,2.242743e+11,4.144481e+07,"MULTIPOLYGON (((-8208500.834 -5733817.475, -82..."


In [26]:
type(regiones)

geopandas.geodataframe.GeoDataFrame

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [27]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

alt.Chart(...)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [28]:
region_names1 = {
    'Aysén' : 'Aysén del General Carlos Ibáñez del Campo',
    'O’Higgins' : "Libertador General Bernardo O'Higgins",
    'Araucanía' : 'La Araucanía',
    'Magallanes' : 'Magallanes y de la Antártica Chilena',
    'Metropolitana' : 'Metropolitana de Santiago',
}

region_names2 = {
    'Región Metropolitana de Santiago' : 'Metropolitana de Santiago',
    'Región de Antofagasta' : 'Antofagasta',
    'Región de Arica y Parinacota' : 'Arica y Parinacota',
    'Región de Atacama' : 'Atacama',
    'Región de Aysén del Gral.Ibañez del Campo' : 'Aysén del General Carlos Ibáñez del Campo',
    'Región de Coquimbo' :  'Coquimbo',
    'Región de La Araucanía' : 'La Araucanía',
    'Región de Los Lagos' : 'Los Lagos',
    'Región de Los Ríos' : 'Los Ríos',
    'Región de Magallanes y Antártica Chilena' : 'Magallanes y de la Antártica Chilena',
    'Región de Tarapacá' : 'Tarapacá',
    'Región de Valparaíso' : 'Valparaíso',
    'Región de Ñuble' : 'Ñuble',
    'Región del Bío-Bío' : 'Biobío',
    "Región del Libertador Bernardo O'Higgins" : "Libertador General Bernardo O'Higgins",
    'Región del Maule' : 'Maule'
}

In [29]:
#-----------------------------------------------------------------------------------
#----------------------------------df Confirmados-----------------------------------
#-----------------------------------------------------------------------------------

df_confirmados = confirmados.drop(['provincia_id', 'provincia', 'comuna_id', 'comuna'], axis=1)
df_confirmados = confirmados.groupby(['region', 'region_id', 'fecha']).agg(poblacion=("poblacion", lambda x: x.sum())).reset_index()

casos_geo = pd.merge(df_confirmados,
                     confirmados.groupby(['region', 'region_id', 'fecha'])['tasa'].sum().reset_index(),
                     on = ['region', 'region_id', 'fecha']
                    )

casos_geo = pd.merge(casos_geo,
                     confirmados.groupby(['region', 'region_id', 'fecha'])['casos_confirmados'].sum().reset_index(),
                     on = ['region', 'region_id', 'fecha']
                    )

#-----------------------------------------------------------------------------------
#----------------------------------df Activos---------------------------------------
#-----------------------------------------------------------------------------------

df_activos = activos.drop(['comuna', 'codigo_comuna'], axis=1)
df_activos = activos.groupby(['region', 'codigo_region', 'fecha']).agg(casos_activos=("casos_activos", lambda x: x.sum())).reset_index()

df_activos = df_activos.rename(columns={'codigo_region':'region_id'})
df_activos['region_id'] = df_activos['region_id'].astype(float).to_frame()

casos_geo = pd.merge(casos_geo,
                     df_activos,
                     on = ['region_id', 'fecha']
                    )

casos_geo = casos_geo.drop('region_y', axis=1)
casos_geo = casos_geo.rename(columns={'region_x': 'region'})

#-----------------------------------------------------------------------------------
#----------------------------------df fallecidos------------------------------------
#-----------------------------------------------------------------------------------

df_fallecidos = fallecidos.drop(fallecidos[fallecidos['region'] == 'Total'].index).reset_index().drop('index', axis=1)
df_fallecidos = df_fallecidos.replace(region_names1)


casos_geo = pd.merge(casos_geo,
                     df_fallecidos,
                     on = ['region', 'fecha']
)

#-----------------------------------------------------------------------------------
#----------------------------------df regiones--------------------------------------
#-----------------------------------------------------------------------------------

df_regiones = regiones.drop(['objectid', 'cir_sena', 'codregion', 'area_km', 'st_area_sh', 'st_length_'], axis=1)
df_regiones = df_regiones.rename(columns={'Region': 'region'})
df_regiones = df_regiones.replace(region_names2)

casos_geo = pd.merge(casos_geo,
                     df_regiones,
                     on = 'region'
                    )

Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [30]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [31]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = casos_geo.loc[lambda x: x['fecha'] == fecha]
    
    chart = alt.Chart(data).mark_geoshape().encode(
        color = col
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [32]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

alt.Chart(...)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [33]:
import ipywidgets as widgets
from ipywidgets import interactive, interact

In [34]:
col_widget = widgets.Dropdown(
    options= [
        'poblacion',
        'casos_confirmados',
        'tasa',
        'casos_activos',
        'fallecidos'
    ],
    description = 'Columna',
    continuous_update = False
)

In [35]:
fecha_widget = widgets.DatePicker(
    description = 'Fecha',
    continuous_update = False
)

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [36]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

interactive(children=(DatePicker(value=None, description='Fecha'), Dropdown(description='Columna', options=('p…

**Comentarios:** Entregar la información de esta forma es muy util y muy visual para el receptor, pero los únicos problema son:
* La gran demora de la maquina para la fabricación de este mapa.
* El hecho de que nuestra data no tiene registros de todos los dias (hay días donde no se subieron registros). 

Aun así podemos extraer mucha información de esto y podemos saber rapidamente que regiones del pais se han visto más afectadas por el Covid-19.